In [1]:
from sage.all import *

# TL;DR
Basically the idea is to find all triples `(a, b, c)` such that `outputs[a], outputs[b], outputs[c]` correspond to `states[i+1], states[i+397], states[i+624]`, link all of them together to create a system of linear equations, then solve to get the original state.

# Setup
There's some mathematical objects related to Python's `random` algorithm we need to setup first.

## The algorithm
The algorithm is MT19937, a LFSR.

Internally, its states are represented by a `624`-item array of `32`-bit integers, but we can represent the states as one `19937`-`GF(2)` vector *(hence that's why the algorithm is called MT19937)*. The story to why it's `19937` but not `19968 = 624*32`, well, I'm lazy and don't know how to explain, but long story short, the state update mechanism doesn't care about the last `31` of `states[0]`, so we're ended up in a structure of `19968 - 31 = 19937` bits. Here's the function to convert state array into a `19937`-`GF(2)` vector:
```py
# Convert MT19937's 624 states
# to 19937 GF(2)-vector
def mt19937_vec_from_624_states(states):
    v = vector(GF(2), 19937)
    v[0] = states[0] >> 31
    
    j = 1
    for i in range(1, 624):
        s = states[i]
        for _ in range(32):
            v[j] = s & 0x1
            s >>= 1
            j += 1
    
    return v
```

Let's call the vector representing the states $\overrightarrow{s}$. 
<br/>
Since it's a LFSR *(the guy I idol so much said this is probably 32 LFSRs combined, so idk...)*, there's a `19937x19937`-`GF(2)` matrix $M$ such that multiplying $M$ with $\overrightarrow{s}$ ($M\overrightarrow{s}$) yields the next MT19937 state.

When we call `random.getrandbits(32)`, we take away the `[1:32]` *(it's `0`-index btw)* column of this vector and perform a `temper()` operation with it before the final result is outputted to the user. This operation, luckily, can be represented as a multiplication with a `32x32` matrix $T$.

Although we're speaking in terms of vectors and multiplication of vectors, since `GF(2)`-operations can be represented as mere `XOR`, `AND`, bitshift, etc... operating on bits, when we view source code, we only see integers shifting around and mixing with each other :-\)

In [2]:
# MT19937 LFSR matrix
M = zero_matrix(GF(2), 19937, 19937)
a = list(map(int,f'{0x9908B0DF:032b}'[::-1]))
for i in range(19937 - 32):
    M[i, i+32] = 1
for i in range(32):
    M[-32+i, 397*32-31+i] = 1
for i in range(32-2):
    M[-32+i, i+2] = 1
for i in range(32):
    M[-32+i, 1] = a[i]
M[-2, 0] = 1

# Temper matrix
T    = load("./T.sobj")
Tinv = T**-1

In [3]:
def temper(y: int) -> int:
    y = y ^ ((y >> 11) & 0xFFFFFFFF)
    y = y ^ ((y <<  7) & 0x9D2C5680)
    y = y ^ ((y << 15) & 0xEFC60000)
    y = y ^  (y >> 18)
    return y

# Utilities
Some utilities to make our lives simpler :-)

In [4]:
from bisect import bisect_left

# Convert l-bit integer to 
# GF(2) l-column vector
def int_to_vec(state, l=32):
    v = []
    for _ in range(l):
        v.append(state & 0x1)
        state >>= 1
    return vector(GF(2), v)

# Convert vector back to int
def vec_to_int(v):
    return int(''.join(map(str, v))[::-1], 2)

# To search value in a sorted list
def findsorted(alist, item):
    i = bisect_left(alist, item)
    if i != len(alist) and alist[i] == item:
        return i
    return None

# Step 1: Build Profile

In lower level code, `random.randrange(0, NN)` just repeatedly calls `random.getrandbits(LL)` until the result `< NN`, where `2^LL` is the closest power of 2 `> NN`.
<br/>
The following function tries to find the range distance between the outputs produced by `states[i+1] & states[i+397]` and `states[i+624] & states[i+397]` to be used in `random.randrange(0, NN)`.

The profiling process helps us narrow down the possibilities to check whether 3 random items in the output array belongs to `states[i+1], states[i+397] and states[i+624]` or not.

The output shows us that:
- `states[i+1] & states[i+397]` are often `[285, 347]` items away from each other
- `states[i+624] & states[i+397]` are often `[158, 203]` items away from each other

In [5]:
import os
import random

def build_profile(NN: int) -> tuple[int, int, int, int]:
    LL = NN.bit_length()

    dist_397_1s   = [] # Collect distances between state[i+397] and states[i+1]   in random.randrange(0, NN) output
    dist_397_624s = [] # Collect distances between state[i+397] and states[i+624] in random.randrange(0, NN) output

    for nsamples in range(100): 
        seed = os.urandom(69)
        randLL = random.Random()
        randLL.seed(seed)
        randNN = random.Random()
        randNN.seed(seed)

        outsLL = [randLL.getrandbits(LL) for i in range(0x10000)] # randomize 0x10000 getrandbits outputs
        outsNN = [randNN.randrange(0,NN) for i in range(0x1337)]  # randomize 0x10000 randrange outputs

        i_original = []
        i = 0
        for outNN in outsNN:
            while outsLL[i] != outNN:
                i += 1
            i_original.append(i)

        for i397, _397 in enumerate(i_original):
            i624 = findsorted(i_original, _397 - (397 - 624)) 
            i1   = findsorted(i_original, _397 - (397 -   1)) 
            if not i624 or not i1:
                continue
            dist_397_1s.append(i397 - i1)
            dist_397_624s.append(i624 - i397)

    return (
        min(dist_397_1s), max(dist_397_1s),
        min(dist_397_624s), max(dist_397_624s)
    )

min_dist_397_1, max_dist_397_1, min_dist_397_624, max_dist_397_624 = build_profile(int(0x13371337*1.337))
print(f'{min_dist_397_1   = }')
print(f'{max_dist_397_1   = }')
print(f'{min_dist_397_624 = }')
print(f'{max_dist_397_624 = }')

min_dist_397_1   = 285
max_dist_397_1   = 347
min_dist_397_624 = 158
max_dist_397_624 = 203


# Step 2: Recovering states and their positions.

Given 3 random outputs `outputs[a], outputs[b] and outputs[c]` for random `a`, `b`, `c`, 
<br/>
how can we figure out that these 3 outputs correspond to the 3 inner states represented by `states[i+1], states[i+397], states[i+624]` or not?

## Notation
First, let's denote:
- vectors $\overrightarrow{s_i}$ corresponds to the values in `states` represented by 32-column `GF(2)` vectors.
- vectors $\overrightarrow{r_i}$ corresponds to the outputs produced from the `states` after going through the `temper` function.
- $T$ is the `32x32-GF(2)` matrix represented by the `temper` function that $T \cdot \overrightarrow{s_i} = \overrightarrow{r_i}$.
- $\overrightarrow{a}$ is the `GF(2)`-vector representing the `0x9908B0DF` constant.

## The math
From the relationship between the states, we derive to the following equation:
$$
    \overrightarrow{s_{i+01}} + \{\overrightarrow{a}, 0\} + \overrightarrow{s_{i+m}} = \overrightarrow{s_{i+n}}
$$
where $\overrightarrow{s_{i+01}}$ represents `(states[0] & 0x80000000 | states[1] & 0x7fffffff) >> 1`.

<br/>

Multiply both sides by $T$:
$$
    T \cdot \overrightarrow{s_{i+01}} + \{T \cdot \overrightarrow{a}, 0\} + \overrightarrow{r_{i+m}} = \overrightarrow{r_{i+n}}
$$

<br/>

`random.randrange` uses `random.getrandbits(LL)`, which takes `LL`**-MSB** bits from the 32-bit states after `temper`. Taking any `LL`**-MSB** bits corresponds to taking the `32-LL -> 31` columns from any `GF(2)` vector.
<br/>
=> Having `outputs[b], outputs[c]` means $T * \overrightarrow{s_{i+01}}$ can be computed partially, so we have an equation involving `states[i+1]` bits.
<br/>
Somewhere else, given `outputs[a]`, we can obtain partial bits of $\overrightarrow{r_{i+1}}$, which also corresponds to another equation involving `states[i+1]` :-)

If the solution from these 2 equations yields an answer for `states[i+1]`, we determine that `outputs[a], outputs[b] & outputs[c]` was indeed produced from `states[i+1], states[i+397], states[i+624]` for any given `i`.

## Terms

The tuple `(a, b, c)` corresponds to some `states[i+1], states[i+397], states[i+624]` for any given `i`, is called a **relationship**.

## Tests

Through trials and errors, it seems like for `NN` with `>= 24` bits, this method would work with little false positives. In `>= 29` bits, near zero false positives has been observed. I don't have the time to map them all out, so you'll have to just take this with a grain of salt ;-\) *(sorry)*.

In [6]:
# Yeah i try to optimize some stuffs
# so this looks really messy af

from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace

__global_cache__ = {}

def build_cache():
    oa = T * int_to_vec(0x9908B0DF)
    V = VectorSpace(GF(2), 31)

    global __global_cache__
    __global_cache__["dgv"] = {}

    for L in range(16, 32):
        TL = T[-L:]
        kerTL = TL.right_kernel()

        W0 = V.subspace([ b.list()[ :-2] + b.list()[-1:] for b in kerTL.basis() ])
        W1 = W0
        WP = V.subspace([ b.list()[1:-1] + [0]           for b in kerTL.basis() ])

        __global_cache__["dgv"][L] = {
            "oaL": oa[-L:],
            "TL": TL,
            "W0": W0,
            "W1": W1,
            "WP": WP,
        }
build_cache()

def try_recover_state_i1(o1, o397, o624):
    L = len(o1)
    assert len(o1) == len(o397) == len(o624) == L

    # Load from cache, no need fancy stuffs
    _c_ = __global_cache__["dgv"][L]
    oaL,TL,W0,W1,WP = _c_["oaL"],_c_["TL"],_c_["W0"],_c_["W1"],_c_["WP"]

    try:
        sol0 = TL.solve_right(o624 + o397).list()
    except:
        sol0 = None
    try:
        sol1 = TL.solve_right(o624 + o397 + oaL).list()
    except:
        sol1 = None
    try:
        solp = TL.solve_right(o1).list()
    except Exception as e:
        raise ValueError("What the fuck?", e)

    if sol1 == None and sol0 == None:
        return
    if sol1 == None and sol0 != None:
        sol1, sol0 = sol0, sol1
    if sol0 == None:
        raise ValueError("interesting")

    v0 = vector(GF(2), sol0[ :-2] + sol0[-1:])
    v1 = vector(GF(2), sol1[ :-2] + sol1[-1:])
    vp = vector(GF(2), solp[1:-1] + [0])

    SS0 = AffineSubspace(v0, W0)
    SS1 = AffineSubspace(v1, W1)
    SSP = AffineSubspace(vp, WP)

    if (S := SS1.intersection(SSP)) is not None:
        s1_0 = 1
    elif (S := SS0.intersection(SSP)) is not None:
        s1_0 = 0
    else:
        return

    s1_1_30_cs_v = S.point()
    for s1_1_30_cs_w in S.linear_part():
        s1_1_30_cs = s1_1_30_cs_v + s1_1_30_cs_w
        s1_1_30,cs = s1_1_30_cs[:-1].list(), s1_1_30_cs[-1]
        if cs != 0:
            continue 

        for s1_31 in (0, 1):
            s1 = [s1_0] + s1_1_30 + [s1_31]
            if vec_to_int(o1) == temper(vec_to_int(s1)) >> (32-L):
                return vector(GF(2), s1)

    return

# Step 3: Building a network.

Notice that sometimes this `states[i+397]` can be other `states[j+1]` or `states[j+624]`. 
<br/>
We can repeatly link different triples *(or relationships)* together using this relevation, over and over again, until it forms a network that spans across hundreds and thousands of states.

Then, we can build a linear solving system to be able to fully solve the initial states :-)

When I test values, it seems like the closer `NN` is to its nearest power of 2, `2^LL`, more relationships are found. 
<br/>
This is true since `random.getrandbits(LL)` are called until its output is `< NN`. If the outputs of `getrandbits` are randomly distributed, then the possibility of one of them also being the `randrange` output is $\frac{NN}{2^{LL}}$. The lowest this value can get is `50%`.

In this challenge, I choose `0x13371337*1.337` as `NN` so `80%` outputs from `getrandbits(LL:=29)` will be used for `randrange(0, NN)`. I tried to go for `50%`, but the relationships found are too sparse, so they couldn't form a large-enough network no matter how many `randrange` outputs I give them. They always end in chunks of `3` or `5`, best are `50`, I think (?) In the end, I don't have the time to test them, so I just leave it at it.

In [7]:
import itertools

# Finds first index tuple (a, b, c) that
#     outputs[a],  outputs[b],    outputs[c]
# corresponds to 3 states variables:
#     states[i+1], states[i+397], states[i+624]
def first_relationship(o, NN) -> tuple[int, int, int]:
    min_dist_397_1, max_dist_397_1, min_dist_397_624, max_dist_397_624 = build_profile(NN)
    for i397 in range(len(o)):
        i1min,   i1max   = max(i397-max_dist_397_1,   0     ), max(i397-min_dist_397_1,   0     )
        i624min, i624max = min(i397+min_dist_397_624, len(o)), min(i397+max_dist_397_624, len(o))

        for i1, i624 in itertools.product(range(i1min, i1max), range(i624min, i624max)):
            if (s1 := try_recover_state_i1(o[i1], o[i397], o[i624])):
                return (i1, i397, i624)

# Using greedy search, find the next tuple (a', b', c') that
#     outputs[a'],  outputs[b'],    outputs[c']
# corresponds to 3 states variables:
#     states[i'+1], states[i'+397], states[i'+624]
# based on the first tuple (a, b, c)
def next_relationship(o, relationship, heuristic=6) -> list[tuple[int, int, int]]:
    _i1, _i397, _i624 = relationship[0]+1, relationship[1]+1, relationship[2]+1
    print(f'({_i1}, {_i397}, {_i624})')

    for i624 in range(_i624, len(o)):
        print('trying', i624)
        for i1, i397 in itertools.product(range(_i1, _i1+heuristic), range(_i397, _i397+heuristic)):
            if (s1 := try_recover_state_i1(o[i1], o[i397], o[i624])):
                return i1, i397, i624
        heuristic += 2 # Should I use a better number?
    
# Combine the above 2 functions
# to find all (a, b, c)
def build_relationships(o, NN):
    relationships = [first_relationship(o, NN)]
    while True:
        new_relationship = next_relationship(o, relationships[-1])
        if new_relationship == None:
            break
        relationships.append(new_relationship)
    return relationships

The below function looks really messy, but it basically creates a function `build_network(i)` that try to build a network that starts from `relationships[i]`.

In [8]:
# Using the relationships gathered from
# the previous outputs, linked them to
# form a network!
def get_build_network_fn(relationships):
    network1s   = [n[0] for n in relationships]
    network397s = [n[1] for n in relationships]
    network624s = [n[2] for n in relationships]

    def set_index(__map__, index, i):
        if index in __map__ and __map__[index] != i:
            raise ValueError("hmm", index, i)
        __map__[index] = i

    def build_network(i: int):
        network = {}
        start = relationships[i]
        queue = [(1, start[0]), (397, start[1]), (624, start[2])]
        while len(queue) > 0:
            index, i = queue.pop()
            try:
                set_index(network, index, i)
            except:
                return {}

            index1   = findsorted(network1s, i)
            index397 = findsorted(network397s, i)
            index624 = findsorted(network624s, i)

            if index1 != None:
                i397 = network397s[index1]
                i624 = network624s[index1]
                if i397 != None and index + 397 - 1 not in network:
                    queue.append((index + 397 - 1, i397))
                if i624 != None and index + 624 - 1 not in network:
                    queue.append((index + 624 - 1, i624))

            if index397 != None:
                i1   = network1s[index397]
                i624 = network624s[index397]
                if i1 != None and index + 1 - 397 not in network:
                    queue.append((index + 1 - 397, i1))
                if i624 != None and index + 624 - 397 not in network:
                    queue.append((index + 624 - 397, i624))

            if index624 != None:
                i1   = network1s[index624]
                i397 = network397s[index624]
                if i1 != None and index + 1 - 624 not in network:
                    queue.append((index + 1 - 624, i1))
                if i397 != None and index + 397 - 624 not in network:
                    queue.append((index + 397 - 624, i397))

        return network
    
    return build_network

# Step 4: Let's crack our challenge!

In [9]:
from output import outputs as outNNs

NN = int(0x13371337*1.337)
LL = NN.bit_length()

# Output converted to vectors
outputs_as_vecs = [ int_to_vec(outNN, LL) for outNN in outNNs ]

This step is annoying as it took me too much time :-\( 
<br/>
The algorithm still works if we try to build relationships from less outputs, but that requires too much experiments to find out which parameters are the best, and unfortunately, I just don't have time to do that :-\(

In [10]:
relationships = build_relationships(outputs_as_vecs, NN)

(1, 315, 499)
trying 499
(2, 316, 500)
trying 500
(3, 317, 501)
trying 501
(4, 318, 502)
trying 502
trying 503
(5, 320, 504)
trying 504
(6, 321, 505)
trying 505
(7, 322, 506)
trying 506
(8, 323, 507)
trying 507
(9, 324, 508)
trying 508
(10, 325, 509)
trying 509
(11, 326, 510)
trying 510
(12, 327, 511)
trying 511
(13, 328, 512)
trying 512
trying 513
(14, 330, 514)
trying 514
(15, 331, 515)
trying 515
(16, 332, 516)
trying 516
(17, 333, 517)
trying 517
trying 518
trying 519
(18, 336, 520)
trying 520
trying 521
(21, 339, 522)
trying 522
(22, 340, 523)
trying 523
(23, 341, 524)
trying 524
(24, 342, 525)
trying 525
(25, 343, 526)
trying 526
(26, 344, 527)
trying 527
trying 528
trying 529
(27, 347, 530)
trying 530
(28, 348, 531)
trying 531
trying 532
(30, 351, 533)
trying 533
(31, 352, 534)
trying 534
(32, 353, 535)
trying 535
(35, 355, 536)
trying 536
(36, 356, 537)
trying 537
trying 538
(38, 357, 539)
trying 539
trying 540
trying 541
(42, 360, 542)
trying 542
trying 543
(44, 361, 544)
tryi

I don't know how to make this optimal, so I tried to build a network from every single relationship. To reduce time, I'll ignore duplicated networks.

In [11]:
from tqdm import trange

build_network = get_build_network_fn(relationships)

started = {}
networks = []
for i in trange(len(relationships)):
    if all(relationships[i][j] in started for j in range(3)):
        continue
    network = build_network(i)
    networks.append(build_network(i))
    for index in network.values():
        started[index] = None

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50415/50415 [00:00<00:00, 66970.13it/s]


In [12]:
# Multiply-by-left of this matrix
# with a state vector advances state
# by 623 rounds
M623 = M**623; print('compute M^623 done')

compute M^623 done


Sometimes, the largest network is not the best network we should choose. 

When I experimented with the `24`-bit case, I've encountered so much fake **relationships**, and it's really hard to categorize them since there's nothing to compare to. So when I try to solve the equations built from these relationships, the program fails. One way I've managed to do is to try to join 2 large networks toghether, and if the join fails, it means that one network contains the bad relationship(s). And use it to categorize & find the best network. I don't think it's an efficient method, but it's the best one I've got right now...

But in the case of `29`-bits, I'm pretty confident that all *(maybe 99.9% ?)* relationships are in high quality :-\)

In [13]:
from sortedcontainers import SortedDict

# Select the biggest network
# we've found and solve from there
network = {}
for _network in networks:
    if len(_network) > len(network):
        network = _network
network = SortedDict(network)

At first, I thought I should just build a simple matrix `S` and stack it up, then use `solve_right()` to recover MT19937's initial state vector. But after a while I've got a `100000`-ish matrix and yes, it's really big, solving takes so much time, so I have to echelonize the matrix occasionally, somehow.

In [14]:
from tqdm import tqdm

S   = Matrix(GF(2), 0, 19938, sparse=False)
TMP = M623*M

S_rank = 0
cache_i = 623
for state_i in (pbar := tqdm(network)):
    assert state_i >= 1
    while state_i > cache_i:
        TMP *= M623
        cache_i += 623

    out_i = network[state_i]
    mat_i = ((state_i-1) % 623)

    out_v = outputs_as_vecs[out_i]
    nTMP  = (T * TMP[1 + mat_i*32 : 1 + mat_i*32 + 32])[-LL:]
    
    S = S.stack(nTMP.augment(out_v))
    if S.dimensions()[0] >= 19937 * 1.5: # 1.5 is a heuristic so that we don't echelonize too much near the end :-)
        S.echelonize(algorithm='m4ri')
        S_rank = S.rank()

        S = S[:S_rank]
        if S_rank == 19937:
            pbar.set_description_str('done')
            break

    pbar.set_description_str(f'S.nrows == {S.dimensions()[0]}, {S_rank = }')

done:  16%|███████████████████                                                                                                   | 10512/65220 [11:45<1:01:13, 14.89it/s]


In [22]:
init_state = S[:, :19937].solve_right(vector(S[:, 19937:].T))

Although you've solved the vector, it's still miss the actual initial state by some rounds, since `random.randrange` skips some unknown amount of rounds at the beginning. That and the first relationship in the network probably misses it by some more. You have to look at the output index of the first relationship in the network to figure out how many states you need to go back as well.

```py
output_index = network[network.keys()[0]]
```

After you go back `output_index` rounds, if `output_index` is not too high, trying to go back `20` rounds more is enough. If it's high, I think you have to try go back further, about `output_index * 0.8`, i think :v

In [23]:
# Multiply-by-left of this matrix
# with a state vector reverses state
# by one round
Minv = M**-1

In [24]:
# Reverse by output_index amount...
output_index = network[network.keys()[0]]
print(f'going back {output_index-1} states...') # minus 1 since the first output is always 1-index

init_state = Minv**(output_index-1) * init_state

going back 0 states...


In [25]:
from cracker.mt19937_crack import RandomSolver
from re import search
from Crypto.Util.number import long_to_bytes

for _ in range(20):
    seed_states = [0x80000000]
    for i in range(1, 624):
        seed_states.append(vec_to_int(init_state[1 + i*32 - 32 : 1 + i*32]))

    # Solve for state to get flag
    solver = RandomSolver()
    solver.init_seed_finder(19968)
    z3_seed_states = solver.get_seed_states()
    solver.solver_constrants.extend([z3_seed_state == seed_state for seed_state, z3_seed_state in zip(seed_states, z3_seed_states)])
    solver.solve()

    # Output flag content
    print(search(b'ISITDTU{[!-z]*}', long_to_bytes(solver.get_seed())))

    # Reverse one round until we find flag
    init_state = Minv * init_state

None
<re.Match object; span=(2158, 2201), match=b'ISITDTU{J1Gs4W-f4ll1ng-1nt0-p14c3-6b3b2203}'>
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


# Conclusion
I really hope that someone out there think of some thing simpler than my dumbass method... I feel like it's too complicated for its own sake...
<br/>
If you feel this is too convoluted to understand, don't be afraid to reach me at my Discord ^.^ I want to thank you for having read through all this mess!